<a href="https://colab.research.google.com/github/Alexksksss/AHP_Group/blob/main/AHP_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [1]:
import numpy as np

In [3]:
!pip install fuzzy-c-means

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.20.0
    Uninstalling typer-0.20.0:
      Successfully uninstalled typer-0.20.0
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.49.1 requires typer<1.0,>=0.12, but you have typer 0.9.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires

In [3]:
from fcmeans import FCM

# ПРИМЕР 1

## ФАЗА 1


In [23]:
# Входные данные из статьи (пример с 6 экспертами и 4 альтернативами)
data = {
    "alternatives": [
        "A1_typhon",
        "A2_drip_irrigation",
        "A3_subirrigation",
        "A4_center_pivot"
    ],
    "dms": [
        {"id": "DM1", "scores": [0.462, 0.210, 0.191, 0.137]},
        {"id": "DM2", "scores": [0.410, 0.300, 0.150, 0.140]},
        {"id": "DM3", "scores": [0.132, 0.422, 0.279, 0.167]},
        {"id": "DM4", "scores": [0.333, 0.205, 0.259, 0.203]},
        {"id": "DM5", "scores": [0.315, 0.377, 0.069, 0.239]},
        {"id": "DM6", "scores": [0.310, 0.281, 0.207, 0.202]}
    ],
    "parameters": {"alpha": 0.95, "B": 2}
}

# Преобразование данных экспертов в numpy-массив
X = np.array([dm["scores"] for dm in data["dms"]])

# Создание DataFrame для удобного отображения
df_orig = pd.DataFrame(X, index=[dm["id"] for dm in data["dms"]], columns=data["alternatives"])
print("Исходные приоритеты экспертов по альтернативам:")
df_orig


Исходные приоритеты экспертов по альтернативам:


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM1,0.462,0.210,0.191,0.137
DM2,0.410,0.300,0.150,0.140
DM3,0.132,0.422,0.279,0.167
DM4,0.333,0.205,0.259,0.203
DM5,0.315,0.377,0.069,0.239
DM6,0.310,0.281,0.207,0.202


In [7]:
# Кластеризация экспертов

# Выбор числа кластеров
n_clusters = 3

# Инициализация и обучение fuzzy c-means
fcm = FCM(n_clusters=n_clusters)
fcm.fit(X)

# Получение степеней принадлежности каждого эксперта к кластерам
membership = fcm.u

# Создание DataFrame для отображения результатов кластеризации
cluster_df = pd.DataFrame(membership, index=[dm["id"] for dm in data["dms"]],
                          columns=[f"Cluster {i+1}" for i in range(n_clusters)])

print("Членство экспертов в кластерах (степени принадлежности):")
cluster_df.round(3)


Членство экспертов в кластерах (степени принадлежности):


,Cluster 1,Cluster 2,Cluster 3
DM1,0.854,0.042,0.104
DM2,0.730,0.051,0.219
DM3,0.000,0.999,0.000
DM4,0.799,0.080,0.121
DM5,0.010,0.005,0.985
DM6,0.656,0.102,0.242


In [11]:
clusters_values = {col: [] for col in cluster_df.columns}
for idx, row in cluster_df.iterrows():
  max_cluster = row.idxmax()
  clusters_values[max_cluster].append(idx)
print(clusters_values)

# Сходится с таблицей 2

{'Cluster 1': ['DM1', 'DM2', 'DM4', 'DM6'], 'Cluster 2': ['DM3'], 'Cluster 3': ['DM5']}


In [22]:
from IPython.display import display

alpha = data["parameters"]["alpha"]
max_iter = 1000
epsilon = 1e-10

def compute_respect_weights(cluster_vectors, alpha):
    n = cluster_vectors.shape[0]
    weights = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist = np.linalg.norm(cluster_vectors[i] - cluster_vectors[j])
            weights[i, j] = np.exp(-alpha * dist)
    return weights

def consensus_within_cluster(cluster_vectors, cluster_dm_ids, alpha, max_iter=100, epsilon=1e-5):
    P = cluster_vectors.copy()
    for iteration in range(max_iter):
        weights = compute_respect_weights(P, alpha)
        P_new = np.zeros_like(P)
        for i in range(len(P)):
            denom = np.sum(weights[i])
            if denom > 0:
                P_new[i] = np.sum(weights[i][:, None] * P, axis=0) / denom
            else:
                P_new[i] = P[i]
        diff = np.linalg.norm(P_new - P)
        P = P_new
        print(f"Итерация {iteration + 1}: изменение = {diff:.6f}")
        df_iter = pd.DataFrame(P, index=cluster_dm_ids, columns=data["alternatives"])
        display(df_iter.round(4))
        if diff < epsilon:
            print("Достигнута сходимость.")
            break
    return P

# Подозначим функцию для запуска по кластерам с выводом таблиц
def run_consensus_on_clusters(X, cluster_membership, alpha):
    clustered_results = {}
    dms_ids = [dm["id"] for dm in data["dms"]]
    for c in range(cluster_membership.shape[1]):
        # Получаем индексы экспертов с членством выше порога
        member_indices = np.where(cluster_membership[:, c] > 0.5)[0]
        cluster_dm_ids = [dms_ids[i] for i in member_indices]
        cluster_vectors = X[member_indices]
        if len(cluster_vectors) > 0:
            print(f"\nКластер {c+1}, эксперты: {cluster_dm_ids}")
            consensus = consensus_within_cluster(cluster_vectors, cluster_dm_ids, alpha)
            clustered_results[c+1] = (cluster_dm_ids, consensus)
        else:
            print(f"\nКластер {c+1} пустой.")
    return clustered_results

# Запускаем для классификации с проверкой
clustered_consensus = run_consensus_on_clusters(X, membership, alpha)



Кластер 1, эксперты: ['DM1', 'DM2', 'DM4', 'DM6']
Итерация 1: изменение = 0.170569


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM1,0.3827,0.2479,0.2007,0.1687
DM2,0.3806,0.2509,0.1995,0.1690
DM4,0.3758,0.2476,0.2042,0.1724
DM6,0.3752,0.2501,0.2025,0.1722


Итерация 2: изменение = 0.008496


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM1,0.3786,0.2491,0.2017,0.1706
DM2,0.3786,0.2491,0.2017,0.1706
DM4,0.3786,0.2491,0.2017,0.1706
DM6,0.3786,0.2491,0.2017,0.1706


Итерация 3: изменение = 0.000024


...

Достигнута сходимость.

Кластер 2, эксперты: ['DM3']
Итерация 1: изменение = 0.000000


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM3,0.132,0.422,0.279,0.167


Достигнута сходимость.

Кластер 3, эксперты: ['DM5']
Итерация 1: изменение = 0.000000


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM5,0.315,0.377,0.069,0.239


Достигнута сходимость.


везде кроме 1 кластера сходится с таблицей 3

## Фаза 2 - межкластерное согласование

In [15]:
#Итеративное согласование между консенсусными векторами кластеров
def consensus_between_clusters(clustered_results, alpha, max_iter=100, epsilon=1e-5):
    # Получаем начальные консенсусные векторы кластеров
    cluster_vectors = np.array([val[1].mean(axis=0) for val in clustered_results.values()])
    cluster_ids = list(clustered_results.keys())

    P = cluster_vectors.copy()

    for iteration in range(max_iter):
        n = len(P)
        weights = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                dist = np.linalg.norm(P[i] - P[j])
                weights[i, j] = np.exp(-alpha * dist)

        P_new = np.zeros_like(P)
        for i in range(n):
            denom = np.sum(weights[i])
            if denom > 0:
                P_new[i] = np.sum(weights[i][:, None] * P, axis=0) / denom
            else:
                P_new[i] = P[i]

        diff = np.linalg.norm(P_new - P)
        P = P_new

        print(f"Итерация {iteration + 1} (межкластерное): изменение = {diff:.6f}")

        df_iter = pd.DataFrame(P, index=[f"Cluster {c}" for c in cluster_ids], columns=data["alternatives"])
        display(df_iter.round(4))

        if diff < epsilon:
            print("Достигнута межкластерная сходимость.")
            break

    # Итоговый групповой консенсус
    final_consensus = np.mean(P, axis=0)
    df_final = pd.DataFrame(final_consensus.reshape(1, -1), columns=data["alternatives"])
    print("Итоговый групповой консенсус по альтернативам:")
    display(df_final.round(4))

    return final_consensus

# Запускаем межкластерное согласование
final_group_consensus = consensus_between_clusters(clustered_consensus, alpha)


Итерация 1 (межкластерное): изменение = 0.245483


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
Cluster 1,0.2873,0.3397,0.1816,0.1914
Cluster 2,0.2600,0.3574,0.1929,0.1898
Cluster 3,0.2818,0.3495,0.1725,0.1963


Итерация 2 (межкластерное): изменение = 0.028043


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
Cluster 1,0.2765,0.3488,0.1822,0.1925
Cluster 2,0.2762,0.3490,0.1824,0.1924
Cluster 3,0.2765,0.3488,0.1822,0.1925


Итерация 3 (межкластерное): изменение = 0.000347


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
Cluster 1,0.2764,0.3489,0.1823,0.1925
Cluster 2,0.2764,0.3489,0.1823,0.1925
Cluster 3,0.2764,0.3489,0.1823,0.1925


Итерация 4 (межкластерное): изменение = 0.000000


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
Cluster 1,0.2764,0.3489,0.1823,0.1925
Cluster 2,0.2764,0.3489,0.1823,0.1925
Cluster 3,0.2764,0.3489,0.1823,0.1925


Достигнута межкластерная сходимость.
Итоговый групповой консенсус по альтернативам:


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
0,0.2764,0.3489,0.1823,0.1925


In [16]:
print("Отчет по результатам двухфазного алгоритма:")

# Таблица исходных оценок
print("\n1) Исходные приоритеты экспертов по альтернативам:")
display(df_orig.round(4))

# Таблица принадлежности кластерам
print("\n2) Членство экспертов в кластерах (степени принадлежности):")
display(cluster_df.round(3))

# Итоговые приоритеты после внутри кластерного консенсуса (по каждому кластеру)
print("\n3) Консенсусные векторы внутри кластеров (последняя итерация):")
for c, (dm_ids, consensus) in clustered_consensus.items():
    print(f"\nКластер {c}, эксперты: {dm_ids}")
    df_c = pd.DataFrame(consensus, index=dm_ids, columns=data["alternatives"])
    display(df_c.round(4))

# Итоговый групповой консенсус по всем кластерам
print("\n4) Итоговый групповой консенсус по альтернативам:")
df_final_consensus = pd.DataFrame(final_group_consensus.reshape(1, -1), columns=data["alternatives"])
display(df_final_consensus.round(4))


Отчет по результатам двухфазного алгоритма:

1) Исходные приоритеты экспертов по альтернативам:


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM1,0.462,0.210,0.191,0.137
DM2,0.410,0.300,0.150,0.140
DM3,0.132,0.422,0.279,0.167
DM4,0.333,0.205,0.259,0.203
DM5,0.315,0.377,0.069,0.239
DM6,0.310,0.281,0.207,0.202



2) Членство экспертов в кластерах (степени принадлежности):


,Cluster 1,Cluster 2,Cluster 3
DM1,0.854,0.042,0.104
DM2,0.730,0.051,0.219
DM3,0.000,0.999,0.000
DM4,0.799,0.080,0.121
DM5,0.010,0.005,0.985
DM6,0.656,0.102,0.242



3) Консенсусные векторы внутри кластеров (последняя итерация):

Кластер 1, эксперты: ['DM1', 'DM2', 'DM4', 'DM6']


...

,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM3,0.132,0.422,0.279,0.167



Кластер 3, эксперты: ['DM5']


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
DM5,0.315,0.377,0.069,0.239



4) Итоговый групповой консенсус по альтернативам:


,A1_typhon,A2_drip_irrigation,A3_subirrigation,A4_center_pivot
0,0.2764,0.3489,0.1823,0.1925


# Пример 2

## Фаза 1

In [24]:
# Входные данные из статьи (пример с 6 экспертами и 4 альтернативами)
data = {
  "alternatives": [
    "IRR",
    "DR",
    "UW",
    "IS",
    "OP"
  ],
  "criteria": [
    {
      "name": "Criterion 1",
      "type": "positive"
    }
  ],
  "dms": [
    {
      "id": "DM1",
      "scores": [
        0.244,
        0.225,
        0.224,
        0.268,
        0.038
      ]
    },
    {
      "id": "DM2",
      "scores": [
        0.240,
        0.319,
        0.253,
        0.149,
        0.039
      ]
    },
    {
      "id": "DM3",
      "scores": [
        0.191,
        0.195,
        0.362,
        0.210,
        0.042
      ]
    },
    {
      "id": "DM4",
      "scores": [
        0.245,
        0.281,
        0.280,
        0.152,
        0.042
      ]
    },
    {
      "id": "DM5",
      "scores": [
        0.354,
        0.249,
        0.216,
        0.140,
        0.041
      ]
    },
    {
      "id": "DM6",
      "scores": [
        0.302,
        0.208,
        0.265,
        0.168,
        0.057
      ]
    },
    {
      "id": "DM7",
      "scores": [
        0.363,
        0.243,
        0.219,
        0.134,
        0.040
      ]
    },
    {
      "id": "DM8",
      "scores": [
        0.278,
        0.328,
        0.158,
        0.177,
        0.058
      ]
    },
    {
      "id": "DM9",
      "scores": [
        0.278,
        0.190,
        0.278,
        0.212,
        0.042
      ]
    },
    {
      "id": "DM10",
      "scores": [
        0.292,
        0.381,
        0.125,
        0.164,
        0.038
      ]
    },
    {
      "id": "DM11",
      "scores": [
        0.251,
        0.413,
        0.116,
        0.173,
        0.048
      ]
    },
    {
      "id": "DM12",
      "scores": [
        0.246,
        0.268,
        0.129,
        0.316,
        0.041
      ]
    }
  ],
  "parameters": {
    "alpha": 0.95,
    "B": 2
  }
}


# Преобразование данных экспертов в numpy-массив
X = np.array([dm["scores"] for dm in data["dms"]])

# Создание DataFrame для удобного отображения
df_orig = pd.DataFrame(X, index=[dm["id"] for dm in data["dms"]], columns=data["alternatives"])
print("Исходные приоритеты экспертов по альтернативам:")
df_orig


Исходные приоритеты экспертов по альтернативам:


,IRR,DR,UW,IS,OP
DM1,0.244,0.225,0.224,0.268,0.038
DM2,0.240,0.319,0.253,0.149,0.039
DM3,0.191,0.195,0.362,0.210,0.042
DM4,0.245,0.281,0.280,0.152,0.042
DM5,0.354,0.249,0.216,0.140,0.041
DM6,0.302,0.208,0.265,0.168,0.057
DM7,0.363,0.243,0.219,0.134,0.040
DM8,0.278,0.328,0.158,0.177,0.058
DM9,0.278,0.190,0.278,0.212,0.042
DM10,0.292,0.381,0.125,0.164,0.038


In [25]:
# Кластеризация экспертов

# Выбор числа кластеров
n_clusters = 6

# Инициализация и обучение fuzzy c-means
fcm = FCM(n_clusters=n_clusters)
fcm.fit(X)

# Получение степеней принадлежности каждого эксперта к кластерам
membership = fcm.u

# Создание DataFrame для отображения результатов кластеризации
cluster_df = pd.DataFrame(membership, index=[dm["id"] for dm in data["dms"]],
                          columns=[f"Cluster {i+1}" for i in range(n_clusters)])

print("Членство экспертов в кластерах (степени принадлежности):")
cluster_df.round(3)


Членство экспертов в кластерах (степени принадлежности):


,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6
DM1,0.249,0.341,0.114,0.062,0.140,0.096
DM2,0.010,0.023,0.014,0.020,0.912,0.022
DM3,0.000,0.000,0.999,0.000,0.000,0.000
DM4,0.012,0.052,0.028,0.016,0.860,0.032
DM5,0.000,0.001,0.000,0.001,0.001,0.996
DM6,0.027,0.646,0.052,0.023,0.097,0.154
DM7,0.001,0.005,0.001,0.002,0.004,0.987
DM8,0.099,0.079,0.035,0.489,0.169,0.129
DM9,0.007,0.944,0.018,0.004,0.015,0.014
DM10,0.012,0.008,0.004,0.947,0.015,0.014


In [26]:
clusters_values = {col: [] for col in cluster_df.columns}
for idx, row in cluster_df.iterrows():
  max_cluster = row.idxmax()
  clusters_values[max_cluster].append(idx)
print(clusters_values)

# Сходится с таблицей 6

{'Cluster 1': ['DM12'], 'Cluster 2': ['DM1', 'DM6', 'DM9'], 'Cluster 3': ['DM3'], 'Cluster 4': ['DM8', 'DM10', 'DM11'], 'Cluster 5': ['DM2', 'DM4'], 'Cluster 6': ['DM5', 'DM7']}


In [27]:
from IPython.display import display

alpha = data["parameters"]["alpha"]
max_iter = 1000
epsilon = 1e-10

def compute_respect_weights(cluster_vectors, alpha):
    n = cluster_vectors.shape[0]
    weights = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist = np.linalg.norm(cluster_vectors[i] - cluster_vectors[j])
            weights[i, j] = np.exp(-alpha * dist)
    return weights

def consensus_within_cluster(cluster_vectors, cluster_dm_ids, alpha, max_iter=100, epsilon=1e-5):
    P = cluster_vectors.copy()
    for iteration in range(max_iter):
        weights = compute_respect_weights(P, alpha)
        P_new = np.zeros_like(P)
        for i in range(len(P)):
            denom = np.sum(weights[i])
            if denom > 0:
                P_new[i] = np.sum(weights[i][:, None] * P, axis=0) / denom
            else:
                P_new[i] = P[i]
        diff = np.linalg.norm(P_new - P)
        P = P_new
        print(f"Итерация {iteration + 1}: изменение = {diff:.6f}")
        df_iter = pd.DataFrame(P, index=cluster_dm_ids, columns=data["alternatives"])
        display(df_iter.round(4))
        if diff < epsilon:
            print("Достигнута сходимость.")
            break
    return P

# Подозначим функцию для запуска по кластерам с выводом таблиц
def run_consensus_on_clusters(X, cluster_membership, alpha):
    clustered_results = {}
    dms_ids = [dm["id"] for dm in data["dms"]]
    for c in range(cluster_membership.shape[1]):
        # Получаем индексы экспертов с членством выше порога
        member_indices = np.where(cluster_membership[:, c] > 0.5)[0]
        cluster_dm_ids = [dms_ids[i] for i in member_indices]
        cluster_vectors = X[member_indices]
        if len(cluster_vectors) > 0:
            print(f"\nКластер {c+1}, эксперты: {cluster_dm_ids}")
            consensus = consensus_within_cluster(cluster_vectors, cluster_dm_ids, alpha)
            clustered_results[c+1] = (cluster_dm_ids, consensus)
        else:
            print(f"\nКластер {c+1} пустой.")
    return clustered_results

# Запускаем для классификации с проверкой
clustered_consensus = run_consensus_on_clusters(X, membership, alpha)



Кластер 1, эксперты: ['DM12']
Итерация 1: изменение = 0.000000


,IRR,DR,UW,IS,OP
DM12,0.246,0.268,0.129,0.316,0.041


Достигнута сходимость.

Кластер 2, эксперты: ['DM6', 'DM9']
Итерация 1: изменение = 0.039102


,IRR,DR,UW,IS,OP
DM6,0.2903,0.1992,0.2713,0.1894,0.0497
DM9,0.2897,0.1988,0.2717,0.1906,0.0493


Итерация 2: изменение = 0.001084


...

Итерация 2: изменение = 0.000055


,IRR,DR,UW,IS,OP
DM5,0.3585,0.246,0.2175,0.137,0.0405
DM7,0.3585,0.246,0.2175,0.137,0.0405


Итерация 3: изменение = 0.000000


,IRR,DR,UW,IS,OP
DM5,0.3585,0.246,0.2175,0.137,0.0405
DM7,0.3585,0.246,0.2175,0.137,0.0405


Достигнута сходимость.


# Фаза 2

In [28]:
#Итеративное согласование между консенсусными векторами кластеров
def consensus_between_clusters(clustered_results, alpha, max_iter=100, epsilon=1e-5):
    # Получаем начальные консенсусные векторы кластеров
    cluster_vectors = np.array([val[1].mean(axis=0) for val in clustered_results.values()])
    cluster_ids = list(clustered_results.keys())

    P = cluster_vectors.copy()

    for iteration in range(max_iter):
        n = len(P)
        weights = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                dist = np.linalg.norm(P[i] - P[j])
                weights[i, j] = np.exp(-alpha * dist)

        P_new = np.zeros_like(P)
        for i in range(n):
            denom = np.sum(weights[i])
            if denom > 0:
                P_new[i] = np.sum(weights[i][:, None] * P, axis=0) / denom
            else:
                P_new[i] = P[i]

        diff = np.linalg.norm(P_new - P)
        P = P_new

        print(f"Итерация {iteration + 1} (межкластерное): изменение = {diff:.6f}")

        df_iter = pd.DataFrame(P, index=[f"Cluster {c}" for c in cluster_ids], columns=data["alternatives"])
        display(df_iter.round(4))

        if diff < epsilon:
            print("Достигнута межкластерная сходимость.")
            break

    # Итоговый групповой консенсус
    final_consensus = np.mean(P, axis=0)
    df_final = pd.DataFrame(final_consensus.reshape(1, -1), columns=data["alternatives"])
    print("Итоговый групповой консенсус по альтернативам:")
    display(df_final.round(4))

    return final_consensus

# Запускаем межкластерное согласование
final_group_consensus = consensus_between_clusters(clustered_consensus, alpha)


Итерация 1 (межкластерное): изменение = 0.311478


,IRR,DR,UW,IS,OP
Cluster 1,0.2663,0.2684,0.2228,0.1997,0.0427
Cluster 2,0.2675,0.2637,0.2318,0.1940,0.0429
Cluster 3,0.2637,0.2622,0.2363,0.1951,0.0428
Cluster 4,0.2679,0.2740,0.2214,0.1941,0.0427
Cluster 5,0.2666,0.2675,0.2302,0.1929,0.0427
Cluster 6,0.2703,0.2665,0.2282,0.1923,0.0428


Итерация 2 (межкластерное): изменение = 0.017286


,IRR,DR,UW,IS,OP
Cluster 1,0.267,0.2671,0.2284,0.1947,0.0428
Cluster 2,0.267,0.2670,0.2285,0.1947,0.0428
Cluster 3,0.267,0.2670,0.2285,0.1947,0.0428
Cluster 4,0.267,0.2671,0.2284,0.1947,0.0428
Cluster 5,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 6,0.267,0.2671,0.2285,0.1947,0.0428


Итерация 3 (межкластерное): изменение = 0.000062


,IRR,DR,UW,IS,OP
Cluster 1,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 2,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 3,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 4,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 5,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 6,0.267,0.2671,0.2285,0.1947,0.0428


Итерация 4 (межкластерное): изменение = 0.000000


,IRR,DR,UW,IS,OP
Cluster 1,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 2,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 3,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 4,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 5,0.267,0.2671,0.2285,0.1947,0.0428
Cluster 6,0.267,0.2671,0.2285,0.1947,0.0428


Достигнута межкластерная сходимость.
Итоговый групповой консенсус по альтернативам:


,IRR,DR,UW,IS,OP
0,0.267,0.2671,0.2285,0.1947,0.0428


In [29]:
print("Отчет по результатам двухфазного алгоритма:")

# Таблица исходных оценок
print("\n1) Исходные приоритеты экспертов по альтернативам:")
display(df_orig.round(4))

# Таблица принадлежности кластерам
print("\n2) Членство экспертов в кластерах (степени принадлежности):")
display(cluster_df.round(3))

# Итоговые приоритеты после внутри кластерного консенсуса (по каждому кластеру)
print("\n3) Консенсусные векторы внутри кластеров (последняя итерация):")
for c, (dm_ids, consensus) in clustered_consensus.items():
    print(f"\nКластер {c}, эксперты: {dm_ids}")
    df_c = pd.DataFrame(consensus, index=dm_ids, columns=data["alternatives"])
    display(df_c.round(4))

# Итоговый групповой консенсус по всем кластерам
print("\n4) Итоговый групповой консенсус по альтернативам:")
df_final_consensus = pd.DataFrame(final_group_consensus.reshape(1, -1), columns=data["alternatives"])
display(df_final_consensus.round(4))


Отчет по результатам двухфазного алгоритма:

1) Исходные приоритеты экспертов по альтернативам:


,IRR,DR,UW,IS,OP
DM1,0.244,0.225,0.224,0.268,0.038
DM2,0.240,0.319,0.253,0.149,0.039
DM3,0.191,0.195,0.362,0.210,0.042
DM4,0.245,0.281,0.280,0.152,0.042
DM5,0.354,0.249,0.216,0.140,0.041
DM6,0.302,0.208,0.265,0.168,0.057
DM7,0.363,0.243,0.219,0.134,0.040
DM8,0.278,0.328,0.158,0.177,0.058
DM9,0.278,0.190,0.278,0.212,0.042
DM10,0.292,0.381,0.125,0.164,0.038



2) Членство экспертов в кластерах (степени принадлежности):


,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6
DM1,0.249,0.341,0.114,0.062,0.140,0.096
DM2,0.010,0.023,0.014,0.020,0.912,0.022
DM3,0.000,0.000,0.999,0.000,0.000,0.000
DM4,0.012,0.052,0.028,0.016,0.860,0.032
DM5,0.000,0.001,0.000,0.001,0.001,0.996
DM6,0.027,0.646,0.052,0.023,0.097,0.154
DM7,0.001,0.005,0.001,0.002,0.004,0.987
DM8,0.099,0.079,0.035,0.489,0.169,0.129
DM9,0.007,0.944,0.018,0.004,0.015,0.014
DM10,0.012,0.008,0.004,0.947,0.015,0.014



3) Консенсусные векторы внутри кластеров (последняя итерация):

Кластер 1, эксперты: ['DM12']


...

,IRR,DR,UW,IS,OP
DM2,0.2425,0.3,0.2665,0.1505,0.0405
DM4,0.2425,0.3,0.2665,0.1505,0.0405



Кластер 6, эксперты: ['DM5', 'DM7']


,IRR,DR,UW,IS,OP
DM5,0.3585,0.246,0.2175,0.137,0.0405
DM7,0.3585,0.246,0.2175,0.137,0.0405



4) Итоговый групповой консенсус по альтернативам:


,IRR,DR,UW,IS,OP
0,0.267,0.2671,0.2285,0.1947,0.0428
